In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os

In [2]:
tbl_merchants = pd.read_parquet("../data/tables/tbl_merchants.parquet")

In [3]:
trans_df = pd.concat([
    pd.read_parquet("../data/tables/transactions_20210228_20210827_snapshot/20210228_20210827.parquet"),
    pd.read_parquet("../data/tables/transactions_20210828_20220227_snapshot/20210828_20220227.parquet")])

In [14]:
trans_df['date'] = pd.to_datetime(trans_df['date'])

In [28]:
count_df = trans_df.groupby(
    ["merchant_abn","date"]
)['dollar_value'].count().reset_index().rename(
    columns={"dollar_value":"daily_order"}
)

In [32]:
df = count_df.groupby("merchant_abn")['daily_order'].mean().reset_index().rename(
    columns={"daily_order":"mean_daily_order"})

In [37]:
df = pd.merge(df, count_df.groupby("merchant_abn")['daily_order'].count().reset_index().rename(
    columns={"daily_order":"count_days"}),on=["merchant_abn"],how="inner")

In [40]:
df.sort_values(["count_days","mean_daily_order"],ascending=False)

,merchant_abn,mean_daily_order,count_days
741,24852446429,455.701370,365
3765,86578477987,428.789041,365
2665,64203420245,410.002740,365
1985,49891706470,388.191781,365
1836,46804135891,368.898630,365
...,...,...,...
3749,86201937910,1.000000,1
3768,86621434137,1.000000,1
3779,86868464441,1.000000,1
4324,98082781436,1.000000,1


In [42]:
value_df = trans_df.groupby(
    ["merchant_abn","date"]
)['dollar_value'].mean().reset_index().rename(
    columns={"dollar_value":"daily_value"}
)

In [45]:
df2 = value_df.groupby("merchant_abn")["daily_value"].mean().reset_index()

In [46]:
pd.merge(df,df2,on="merchant_abn",how="inner").sort_values(["count_days","mean_daily_order"],ascending=False)

,merchant_abn,mean_daily_order,count_days,daily_value
0,10023283211,5.104396,364,211.890526
1,10142254217,4.792717,357,38.709905
2,10165489824,1.000000,2,15786.793362
3,10187291046,1.349315,146,118.495731
4,10192359162,1.329268,164,468.631061
...,...,...,...,...
4409,99974311662,1.085714,70,285.468132
4410,99976658299,36.676712,365,149.437813
4411,99987905597,1.123457,81,349.662002
4412,99989036621,1.000000,1,56552.394443
